# How many Polarize puzzles are there?

I want to be sure that there are enough interesting puzzles to set one every day. (Here's is the equivalent [investigation for Reflect](https://github.com/tomwhite/reflect/blob/main/combinatorics.ipynb).)

## Counting Polarize boards

Here is some code to count the total number of boards (of size 4) for various numbers of pieces. The first column counts the number of ways of placing the pieces on a board, without taking board symmetries into account. The second column counts the number of distinct boards, where symmetries are taken into account.

In [1]:
%%time
import pandas as pd
from polarize.encode import all_boards, canonical_boards

rows = []
for num_pieces in range(1, 7):
    boards, _, _ = all_boards(num_pieces)
    distinct_boards, _, _ = canonical_boards(num_pieces)
    rows.append([num_pieces, len(boards), len(distinct_boards)])
pd.DataFrame(
    rows,
    columns=[
        "Number of dominoes",
        "Number of 4x4 boards",
        "Number of distinct 4x4 boards",
    ],
)

CPU times: user 24.5 s, sys: 3.77 s, total: 28.2 s
Wall time: 31 s


,Number of dominoes,Number of 4x4 boards,Number of distinct 4x4 boards
0,1,96,14
1,2,3584,476
2,3,66816,8420
3,4,663808,83343
4,5,3469312,434256
5,6,8806400,1102224


## Counting Polarize puzzles

Some boards are the solution to more than one puzzle - i.e. a given set of lights and dominoes. We don't want to set puzzles that have multiple solutions, so here we look for the number of different puzzles that have a unique solution.

In [2]:
%%time
from polarize.encode import canonical_puzzles_with_unique_solution

rows = []
for num_pieces in range(1, 7):
    boards, _, _ = all_boards(num_pieces)
    distinct_boards, _, _ = canonical_boards(num_pieces)
    canonical_lights, canonical_dominoes = canonical_puzzles_with_unique_solution(
        num_pieces
    )
    rows.append([num_pieces, len(boards), len(distinct_boards), len(canonical_lights)])
pd.DataFrame(
    rows,
    columns=[
        "Number of dominoes",
        "Number of 4x4 boards",
        "Number of distinct 4x4 boards",
        "Number of distinct 4x4 puzzles with a unique solution",
    ],
)

CPU times: user 36 s, sys: 5.49 s, total: 41.5 s
Wall time: 44.8 s


,Number of dominoes,Number of 4x4 boards,Number of distinct 4x4 boards,Number of distinct 4x4 puzzles with a unique solution
0,1,96,14,14
1,2,3584,476,238
2,3,66816,8420,1606
3,4,663808,83343,3161
4,5,3469312,434256,3313
5,6,8806400,1102224,3233


We also have a rule where you have to use *all* the domino pieces in the puzzle. So let's count the number of puzzles which have unique solutions even when considering fewer pieces.

In [9]:
import numpy as np
from polarize.encode import decode_puzzle
from polarize.solve import quick_has_unique_solution


def count_unique_with_fewer(canonical_lights, canonical_dominoes):
    unique_with_fewer = np.empty(canonical_lights.shape, dtype=bool)
    for i in range(len(canonical_lights)):
        puzzle = decode_puzzle(canonical_lights[i], canonical_dominoes[i])
        unique_with_fewer[i] = quick_has_unique_solution(
            puzzle, fewer_pieces_allowed=True
        )
    return np.count_nonzero(unique_with_fewer)

In [12]:
%%time
from polarize.encode import canonical_puzzles_with_unique_solution

rows = []
for num_pieces in range(1, 6):
    boards, _, _ = all_boards(num_pieces)
    distinct_boards, _, _ = canonical_boards(num_pieces)
    canonical_lights, canonical_dominoes = canonical_puzzles_with_unique_solution(
        num_pieces
    )
    unique_with_fewer = count_unique_with_fewer(canonical_lights, canonical_dominoes)
    rows.append(
        [
            num_pieces,
            len(boards),
            len(distinct_boards),
            len(canonical_lights),
            unique_with_fewer,
        ]
    )
pd.DataFrame(
    rows,
    columns=[
        "Number of dominoes",
        "Number of 4x4 boards",
        "Number of distinct 4x4 boards",
        "Number of distinct 4x4 puzzles with a unique solution",
        "Number of distinct 4x4 puzzles with a unique solution including fewer pieces",
    ],
)

CPU times: user 40.8 s, sys: 5.33 s, total: 46.1 s
Wall time: 46.8 s


,Number of dominoes,Number of 4x4 boards,Number of distinct 4x4 boards,Number of distinct 4x4 puzzles with a unique solution,Number of distinct 4x4 puzzles with a unique solution including fewer pieces
0,1,96,14,14,14
1,2,3584,476,238,238
2,3,66816,8420,1606,1515
3,4,663808,83343,3161,1876
4,5,3469312,434256,3313,424


From experience, 4 or 5 dominoes make a challenging puzzle, so we might have a few years of puzzles. (This is a lot less than over 500 years of Reflect puzzles!)

But that's probably OK. We could set puzzles that are transformations of each other (i.e. don't take symmetries into account) as players will probably not notice. And then repeat after they've been exhausted - if people are still playing it.

Of course, another way to increase the number of puzzles would be to use a larger board (5x5) and to have more complex pieces (triominoes).

## All the puzzles

Let's generate all the puzzles (with a given number of dominoes).

Note that the puzzles are *not* canonicalized, so we've got all transformations in this data.

In [3]:
from polarize.encode import all_puzzles_with_unique_solution

boards, lights, dominoes = all_puzzles_with_unique_solution(4)
len(boards), len(lights), len(dominoes)

(25240, 25240, 25240)

### Solving a puzzle

We can use this data to find the solution for any puzzle (that has a unique solution) simply by finding a matching row.

Here's a puzzle set on 10 April 2025:

In [4]:
from polarize.model import Puzzle

puzzle = Puzzle.from_json_str(
    """{"n": 4, "lights": [0, 2, 2, 1, 2, 2, 2, 1], "dominoes": [3, 6, 5, 2], "initial_placed_dominoes": [{"domino": 5, "i": 1, "j": 0}, {"domino": 6, "i": 0, "j": 0}, {"domino": 3, "i": 2, "j": 0}, {"domino": 2, "i": 2, "j": 1}], "solution": {"values": [[0, 0, 0, 0], [2, 2, 1, 0], [1, 1, 2, 2], [0, 2, 0, 0]], "placed_dominoes": [{"domino": 3, "i": 2, "j": 2}, {"domino": 6, "i": 0, "j": 1}, {"domino": 5, "i": 1, "j": 2}, {"domino": 2, "i": 1, "j": 1}]}}"""
)

from rich.jupyter import print

print(puzzle)

████ 
█....█
█....█
█....█
█....█
 ████ 
\\
\
/
/
\
\/

Let's first encode the lights and the dominoes.

In [5]:
from polarize.encode import encode_dominoes

lights_val = puzzle.lights_int
dominoes_val = encode_dominoes(
    np.array([d.value for d in puzzle.dominoes], dtype=np.int8)
)

lights_val, dominoes_val

(10665, 1114384)

The filter the rows to those with matching dominoes

In [6]:
boards_with_dominoes = boards[dominoes == dominoes_val]
lights_with_dominoes = lights[dominoes == dominoes_val]
len(boards_with_dominoes), len(lights_with_dominoes)

(28, 28)

Then filter by lights

In [7]:
matching_boards = boards_with_dominoes[lights_with_dominoes == lights_val]
len(matching_boards)

1

There is a single match! Here's the (decoded) board and the solution from the original puzzle. They are the same.

In [8]:
from polarize.encode import decode_board

print(decode_board(matching_boards[0]))

print(puzzle.solution)

████ 
█....█
█\\/.█
█//\\█
█.\..█
 ████

████ 
█....█
█\\/.█
█//\\█
█.\..█
 ████

### Generating a puzzle

We can pick a row at random and use the lights and dominoes to set a puzzle (which has a unique solution):

In [9]:
i = 10000
puzzle = decode_puzzle(lights[i], dominoes[i])

from rich.jupyter import print

print(puzzle)

████ 
█....█
█....█
█....█
█....█
 ████ 
//
/
\
\
/
\
/

Let's check that the solution is actually unique and matches the board in the `boards` variable.

In [10]:
from polarize.solve import solve

solutions = solve(puzzle)
assert len(solutions) == 1
print(solutions[0])

████ 
█.../█
█//.\█
█.\.\█
█././█
 ████

In [11]:
print(decode_board(boards[i]))

████ 
█.../█
█//.\█
█.\.\█
█././█
 ████